In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import skimage.segmentation as seg
import skimage.filters as filt
import skimage.morphology as morph
import skimage.draw as draw
from scipy import ndimage
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import cumtrapz
import scipy.ndimage.filters as filters
import time

In [3]:
def show(img_path):
  plt.imshow(img_path)
  plt.axis('off')
  plt.show()

  
def count_pos(arr):
  count = 0
  for num in arr:
    if num > 0:
        count += 1

  print("Number of positive elements in the array:", count)


In [4]:
import numpy as np
from scipy.integrate import cumtrapz

def curvature(x, y, xc, yc, r):
    # Shift coordinates to make the center of the circle the origin
    x_shifted = x - xc
    y_shifted = y - yc
    # Calculate distance from each point to the center of the circle
    d = np.sqrt(x_shifted**2 + y_shifted**2)
    # Calculate the curvature only for points inside the circle
    inside_circle = d <= r
    dx_dt = np.gradient(x[inside_circle])
    dy_dt = np.gradient(y[inside_circle])
    d2x_dt2 = np.gradient(dx_dt)
    d2y_dt2 = np.gradient(dy_dt)
    denominator = dx_dt**2 + dy_dt**2
    curvature = np.zeros_like(denominator)
    nonzero_denominator = denominator != 0
    curvature[nonzero_denominator] = (dx_dt[nonzero_denominator] * d2y_dt2[nonzero_denominator] - d2x_dt2[nonzero_denominator] * dy_dt[nonzero_denominator]) / denominator[nonzero_denominator]**(3/2)
    # Pad the curvature array with zeros for the points outside the circle
    curvature_padded = np.zeros_like(d)
    curvature_padded[inside_circle] = curvature
    return curvature_padded


def integral_curvature(x, y, xc, yc, r):
    k = curvature(x, y, xc, yc, r)
    return cumtrapz(k, initial=0)


def get_integral_curvature(i, x, y, r, memo):
    # Check if the result has already been computed
    if (i, r) in memo:
        return memo[(i, r)]
    # Compute the integral curvature for a circle centered at (x[i], y[i]) with radius r
    int_curv = integral_curvature(x, y, x[i], y[i], r)[-1]
    # Memoize the result
    memo[(i, r)] = int_curv
    return int_curv


def get_curvatures(contours, radius_arr):
    curvature_arr = []
    # Loop over each contour
    for contour in contours:
        curr = []
        # Precompute the coordinates of all points in the contour
        x, y = contour[:, 0, 0], contour[:, 0, 1]
        # Compute the area measure for each point in the contour
        memo = {}
        for radius in radius_arr:
            curr_int = []
            for i in range(len(contour)):
                curr_int.append(get_integral_curvature(i, x, y, radius, memo))
            curr.append(curr_int)
        curvature_arr.append(curr)

    return curvature_arr

def to_hist(currvature_arr):
  currvature_arr = np.array(currvature_arr)
  x = np.arange(currvature_arr.shape[1]) # x-axis is the number of contour points
  y = np.arange(currvature_arr.shape[0]) # y-axis is the different scales
  X, Y = np.meshgrid(x, y)
  hist_values = []


In [ ]:
# specify path to the folder containing image files
radius_arr = np.arange(10, 131, 5)
path = "../../leafsnap/leafsnap-dataset/dataset/segmented/field"

counter = 0

# initialize empty lists for features and labels
features = []
labels = []

# loop through all subfolders in the path
for foldername in os.listdir(path):
    
    folderpath = os.path.join(path, foldername)
    if not os.path.isdir(folderpath):
        continue

    if counter == 1:
        break
        
    # loop through all image files in the subfolder
    for filename in os.listdir(folderpath):

        filepath = os.path.join(folderpath, filename)
        if not os.path.isfile(filepath):
            continue
        
        # try:
            
        start_time = time.time()
        
        # Load the image
        img = cv2.imread(filepath)

        # set the new size
        new_size = (250, 250)

        # resize the image
        img = cv2.resize(img, new_size)

        # Convert the image to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Apply a Gaussian blur to the image to reduce noise
        blur = cv2.GaussianBlur(gray, (5,5), 0)

        # Apply Canny edge detection to get the edges
        edges = cv2.Canny(blur, 100, 200)

        # Find the contours in the image
        contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # Merge all the contours
        merged_contours = []
        for contour in contours:
            merged_contours += contour.tolist()

        oc = np.array([merged_contours])
        currvature_arr = get_curvatures(oc,radius_arr)
        curvature_arr = np.array(currvature_arr[0])
        x = np.arange(curvature_arr.shape[1]) # x-axis is the number of contour points
        y = np.arange(curvature_arr.shape[0]) # y-axis is the different scales
        X, Y = np.meshgrid(x, y)

        hist_values = []
        for i in range(curvature_arr.shape[0]):
            counts, bin_edges = np.histogram(curvature_arr[i], bins=21) # adjust the number of bins as needed
            hist_values.append(counts)

        data = np.array([np.array(hist) for hist in hist_values])

        X2 = data.flatten().tolist()

        print(X2)

        features.append(X2)
        labels.append(foldername)

        end_time = time.time()
        total_time = end_time - start_time

        # except:
        #     pass

        counter += 1
        
    print(f"Finished plant: {foldername}")
    
# convert features and labels to numpy arrays
features = features
labels = np.array(labels)

# save features and labels to a pandas dataframe and export to CSV file
data = pd.DataFrame({"hist_values" : features})
data["plant"] = labels
data.to_csv("lf_data.csv", index=False)

In [113]:
radius_arr = np.arange(10, 131, 5)
# Load the image
img = cv2.imread('/content/13002220995650.png')
# set the new size
new_size = (250, 250)
# resize the image
img = cv2.resize(img, new_size)
# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# Apply a Gaussian blur to the image to reduce noise
blur = cv2.GaussianBlur(gray, (5,5), 0)
# Apply Canny edge detection to get the edges
edges = cv2.Canny(blur, 100, 200)
# Find the contours in the image
contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# Merge all the contours
merged_contours = []
for contour in contours:
    merged_contours += contour.tolist()
oc = np.array([merged_contours])

In [114]:

start_time = time.time()
currvature_arr = get_curvatures(oc,radius_arr)
end_time = time.time()
total_time = end_time - start_time
print(f"Total run time: {total_time} seconds")

Total run time: 2.6668825149536133 seconds
